In [1]:
# we'll use sqlite to build a database
import sqlite3 as sql
# we'll use pandas to manipulate and analyze data
import pandas as pd

In [2]:
# set file name
CSV_FILE_PATH = 'WIFI_connections_home_assignment.csv'

In [3]:
# read csv file and preview data
connections = pd.read_csv(CSV_FILE_PATH)
connections.head()

,connection_date,user_id,start_hour,start_minute,end_hour,end_minute
0,20171028,24635,13,18,13,25
1,20171028,28403,8,3,8,38
2,20171028,28723,11,13,11,17
3,20171028,29071,5,47,6,5
4,20171028,29431,11,10,11,33


In [4]:
# check how many rows
connections.shape

(11601, 6)

In [5]:
# take a quick look at the data... we want to see:
# if we have missing values (count)
# hours min/max should be between 0-25, minutes should be between 0-59
connections.describe()

,connection_date,user_id,start_hour,start_minute,end_hour,end_minute
count,1.160100e+04,11601.000000,11601.000000,11601.000000,11601.000000,11601.000000
mean,2.017108e+07,356465.564779,12.177399,28.827170,12.414447,29.512801
std,3.458271e+01,204067.310925,5.085738,17.318593,5.080417,17.350619
min,2.017103e+07,171.000000,0.000000,0.000000,0.000000,0.000000
25%,2.017103e+07,163712.000000,8.000000,13.000000,8.000000,14.000000
50%,2.017110e+07,337862.000000,12.000000,29.000000,12.000000,30.000000
75%,2.017111e+07,544997.000000,17.000000,43.000000,17.000000,45.000000
max,2.017111e+07,714719.000000,23.000000,59.000000,23.000000,59.000000


In [6]:
conn = sql.connect('selina.db')
cursor = conn.cursor()

In [7]:
cursor.execute('CREATE TABLE raw_wifi_connections( connection_date char(8), user_id varchar(25), start_hour int, start_minute int, end_hour int, end_minute int)')


In [8]:
# insert data into raw table
connections.to_sql(name='raw_wifi_connections', con=conn, if_exists='append', index=False)
#check that we inserted the right total # of rows
conn.commit()
countQuery = "select count(*) as record_count from raw_wifi_connections"
cursor.execute(countQuery)
conn_record_count = cursor.fetchall()
conn_record_count

[(11601,)]

In [9]:
# let's create a date dimension table that we can reference the true date format and other attritbutes
# for the sake of time I am just going to use the dates in the file, but a good database should have all date possibilites (past&future)
cursor.execute('CREATE TABLE d_date(date_str char(8), day_date DATE NOT NULL, year_num INT, month_num INT, week_of_year_num INT, day_num INT, day_of_week INT, day_of_week_name varchar(10))')

In [10]:
populate_d_date_SQL="INSERT INTO d_date (date_str, day_date, year_num, month_num, day_num) \
SELECT DISTINCT connection_date, \
	substr(connection_date, 1, 4) || '-' || substr(connection_date, 5, 2)|| '-' || substr(connection_date,7, 2)  AS day_date \
    , substr(connection_date, 1, 4) as year_num \
    , cast(substr(connection_date, 5, 2) as int)  as month_num \
    , cast(substr(connection_date, 7, 2) as int)  as day_num \
FROM raw_wifi_connections"

In [11]:
update_d_date_SQL = "UPDATE d_date  \
SET week_of_year_num=strftime('%W',day_date) \
, day_of_week = cast (strftime('%w', day_date) as integer) \
, day_of_week_name = case cast (strftime('%w', day_date) as integer) \
  when 0 then 'Sunday' \
  when 1 then 'Monday' \
  when 2 then 'Tuesday' \
  when 3 then 'Wednesday' \
  when 4 then 'Thursday' \
  when 5 then 'Friday' \
  else 'Saturday' end"

In [12]:
cursor.execute(populate_d_date_SQL)
cursor.execute(update_d_date_SQL)
#check row count
conn.commit()
countQuery = "select count(*) as record_count, count(week_of_year_num) as woy_count from d_date"
cursor.execute(countQuery)
d_record_count = cursor.fetchall()
d_record_count

[(14, 14)]

In [13]:
# Now we are going to create a table to replicate hours per day so we can identify who is connected at every hour in the viz (instead of just start/end)
hours_in_day=pd.Series(range(0,24), name='hour_int')
print(hours_in_day)

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
Name: hour_int, dtype: int64


In [14]:
hours_in_day.to_sql(name='d_hour', con=conn, if_exists='replace', index=False)
#check row count
conn.commit()
countQuery = "select count(*) as record_count from d_hour"
cursor.execute(countQuery)
d_record_count = cursor.fetchall()
d_record_count

[(24,)]

In [15]:
# Create our reporting fact table
cursor.execute('CREATE TABLE f_wifi_connections(id INTEGER PRIMARY KEY AUTOINCREMENT, user_id varchar(25), connection_date char(8), connection_start_datetime datetime, connection_end_datetime datetime, start_hour int, start_minute int, end_hour int, end_minute int)')

In [18]:
populate_f_SQL="INSERT INTO f_wifi_connections (user_id, connection_date, connection_start_datetime, connection_end_datetime, start_hour, start_minute, end_hour, end_minute) \
SELECT DISTINCT user_id, \
    r.connection_date, \
	d.day_date || ' ' || \
        case when r.start_hour<10 then '0' else '' end || \
        r.start_hour|| ':' || case when r.start_minute<10 then '0' else '' end || r.start_minute   AS start_datetime, \
	d.day_date || ' ' || \
        case when r.end_hour<10 then '0' else '' end || \
        r.end_hour|| ':' || case when r.end_minute<10 then '0' else '' end || r.end_minute  AS end_datetime \
    ,  start_hour, start_minute, end_hour, end_minute \
FROM raw_wifi_connections r \
JOIN d_date d  \
on r.connection_date=d.date_str"

In [19]:
cursor.execute(populate_f_SQL)
#check row count
conn.commit()
countQuery = "select count(*) as record_count from f_wifi_connections"
cursor.execute(countQuery)
d_record_count = cursor.fetchall()
d_record_count

[(11601,)]

In [20]:
conn.commit()

cursor.close()

In [22]:
exportQuery = "select f.user_id \
                , f.connection_start_datetime \
                , f.connection_end_datetime \
                , h.hour_int as active_hour_int \
                , d.*  \
from f_wifi_connections f \
join d_hour h on h.hour_int between f.start_hour and f.end_hour \
left join d_date d on f.connection_date=d.date_str"
connections_export_df=pd.read_sql_query(exportQuery, conn)
connections_export_df.head()

,user_id,connection_start_datetime,connection_end_datetime,active_hour_int,date_str,day_date,year_num,month_num,week_of_year_num,day_num,day_of_week,day_of_week_name
0,24635,2017-10-28 13:18,2017-10-28 13:25,13,20171028,2017-10-28,2017,10,43,28,6,Saturday
1,28403,2017-10-28 08:03,2017-10-28 08:38,8,20171028,2017-10-28,2017,10,43,28,6,Saturday
2,28723,2017-10-28 11:13,2017-10-28 11:17,11,20171028,2017-10-28,2017,10,43,28,6,Saturday
3,29071,2017-10-28 05:47,2017-10-28 06:05,5,20171028,2017-10-28,2017,10,43,28,6,Saturday
4,29071,2017-10-28 05:47,2017-10-28 06:05,6,20171028,2017-10-28,2017,10,43,28,6,Saturday


In [23]:
connections_export_df.to_csv('wifi_connections_tableau.csv')